In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


The filename, directory name, or volume label syntax is incorrect.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from time import time
import re
import pickle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
cleaned_train = pd.read_csv('cleaned_train.csv')

In [4]:
print(cleaned_train["raw_address"][0])
print(cleaned_train["POI"][0])
print(cleaned_train["street"][0])

jl kapuk timur delta sili iii lippo cika 11 a cicau cikarang pusat
nan
jl kapuk timur delta sili iii lippo cika


In [5]:
!pip install spacy --user

In [6]:
import numpy as np
import spacy
from spacy.tokens import Span
from spacy.lang.id import Indonesian
from spacy.training import Example

nlp = Indonesian()
nlp.add_pipe('ner')
nlp.begin_training()

C:\ProgramData\Anaconda3\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [7]:
cleaned_train

,id,raw_address,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a ...,NaN,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",NaN,NaN
2,2,setu siung 119 rt 5 1 13880 cipayung,NaN,siung
3,3,"toko dita, kertosono",toko dita,NaN
4,4,jl. orde baru,NaN,jl. orde baru
...,...,...,...,...
299995,299995,jend ahmad yani 331 kertasari ciamis,NaN,jend ahmad yani
299996,299996,"raya cila kko, cilandak timur kel.",NaN,raya cila kko
299997,299997,tanjung gusta jl. yaya 2 no 17,NaN,NaN
299998,299998,jalan cipadu jaya taman asri gang bijaksana 3 ...,taman asri,NaN


In [8]:
haha = cleaned_train["raw_address"][0]
hehe = re.split(r", |\. |\s|\n", haha)
print(hehe)

['jl', 'kapuk', 'timur', 'delta', 'sili', 'iii', 'lippo', 'cika', '11', 'a', 'cicau', 'cikarang', 'pusat']


In [9]:
street = str(cleaned_train["street"][1])
print(street)
arr = []
if street == "nan":
    arr = re.split(", |\. |\s", street)
    
if arr:
    print(arr)

nan
['nan']


In [10]:
import re

train_data = []
for i in range(cleaned_train["raw_address"].size):
    raw_address = cleaned_train["raw_address"][i]
    # doc = nlp(raw_address)
    address_words = re.split(r", |\. |\s", raw_address)
    street = str(cleaned_train["street"][i])
    street_words = []
    if street != "nan":
        street_words = re.split(r", |\. |\s", street)
        
    poi = str(cleaned_train["POI"][i])
    poi_words = []
    if poi != "nan":
        poi_words = re.split(r", |\. |\s", poi)
    
    data = []
    labels = []
    
    # For each street word
    for i,street_word in enumerate(street_words):
        data.append(street_word)
        label = ""
        if len(street_words) == 1:
            label = "U"
        elif i == 0:
            label = "B"
        elif i == (len(street_words)-1):
            label = "L"
        else:
            label = "I"
        label += "-STREET"
        labels.append(label)
        
    # For each poi word
    for i,poi_word in enumerate(poi_words):
        data.append(poi_word)
        label = ""
        if len(poi_words) == 1:
            label = "U"
        elif i == 0:
            label = "B"
        elif i == (len(poi_words)-1):
            label = "L"
        else:
            label = "I"
        label += "-POI"
        labels.append(label)
    
    # For words not in street or poi
    for word in address_words:
        if word not in data:
            data.append(word)
            labels.append("O")
            
    train_data.append((' '.join(data), labels))

In [11]:
import random

optimizer = nlp.initialize()
for itn in range(30):
    random.shuffle(train_data)
    for raw_text, entity_offsets in train_data:
        doc = nlp.make_doc(raw_text)
        example = Example.from_dict(doc, {"entities":entity_offsets})
        nlp.update([example], sgd=optimizer)

nlp.to_disk("/output")

Iteration  0  done


In [12]:
doc = nlp("habitat terrace residence blok d8, jl..masjid ciater, ciater, serpong, 15317.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('habitat terrace residence', 'POI')]


In [13]:
cleaned_test = pd.read_csv('cleaned_test.csv')
cleaned_test

,id,POI/street
0,0,s. parman 53 sidanegara 4 cilacap tengah
1,1,"anggrek per, baloi indah kel. lubuk baja"
2,2,"asma laun, mangund imog,"
3,3,"ud agung rejeki, raya ngawi- sri wedari karang..."
4,4,"cut mutia, 35 baiturrahman"
...,...,...
49995,49995,toko mbak farid semboro semboro
49996,49996,"vie - tk. ridho kids, vete 3 cari, 16720 ciawi"
49997,49997,"mart dan roti bakar malabar, nasio,"
49998,49998,graha indah pamulang jl. mujair raya bambanbu ...


In [14]:
results = []
for row in cleaned_test["POI/street"]:
    print(row)
    doc = nlp(row)
    poi = ""
    street = ""
    
    for ent in doc.ents:
        if ent.label_ == "POI":
            poi += ent.text
        if ent.label_ == "STREET":
            street += ent.text
    
    result = poi+'/'+street
    print(result)
    results.append(result)
            

s. parman 53 sidanegara 4 cilacap tengah
/s. parman
anggrek per, baloi indah kel. lubuk baja
/anggrek per,
asma laun, mangund imog,
/asma laun,
ud agung rejeki, raya ngawi- sri wedari karanganyar
ud agung/
cut mutia, 35 baiturrahman
/cut mutia,
pem dos dapur ala perumahan gar no a 12 suka jaya sukarami
/pem dos
tb. mara, cisayong
tb./
pura taman beji tista,
pura taman beji/
mangkura tk pam, 90113 ujung pandang
tk pam, 90113 ujung/mangkura
raya wonoso wonotunggal wonotunggal
/raya wonoso
tebet timur tebetung raya 92 rt 2 1 tebet
/tebet timur tebetung raya
kemur viii, no 95 glodok rt 5 rw 5 taman sari
/kemur viii,
kimia farma,
/kimia farma,
war bakso lata kota waingapu
bakso lata kota waingapu/war
toko teddy raya pandan jakat,
toko teddy raya pandan/
slb bina ban, raya sawunggngg
slb bina/
dimas laundry demak, jl tanjung ii, katonsari
dimas laundry/
waroeng baro batik ayu 3, 23
waroeng baro batik ayu/
pasar pagi raya 78 roa malaka rt 1 rw 1 tambora
/pasar pagi raya
dens foto, sukam,
/den

/jatiluhur raya,
gg. djuli, 3 cisarua cikole
/gg.
by pass ngurah rai 46 benoa kuta selatan
/by pass ngurah rai
es campur jus pokat, melur, senapelan
es campur jus/
marga 13 buahbatu (margacinta)
/marga
bida ayu pintu 3 84 mangsang sei sungai beduk
bida ayu pintu 3/
jelambar sela 15,
/jelambar sela
pisangan baru ten 2 pisangan baru rt 5 5 matraman
/pisangan baru ten
surya kencana cv letda suj, bandaruss selamat
surya kencana cv/
wis rah seme pt, jend s parmanman v, waru waru
/wis rah
sumberkah jaya perumahan kota baru indah medan, 52 rt 61
perumahan kota baru indah/sumberkah jaya
bank rakyat indonesiaesia, raya penggilalgas, pengasinan sawangan
bank rakyat/
balilaris pie susu & lapis legit by pass ngurah rai, kuta
balilaris pie susu & lapis legit/
pahl trip oro-oro dowo
trip oro-oro dowo/pahl
manta tour ir. h. jua iii,
manta tour ir/
raya bekasi, 45 1 cakung
/raya bekasi,
toko joven, pasar pagi, 80a 2
toko joven, pasar/
keta kali 71 ketabang 5 genteng
/keta kali
5000 plus toko, pon kela

/menanggal menangka
ken 3 pekayon jaya kel.
/ken 3
haji m joni, no 151 medan area
/haji m
rm. pad, raya ciboree,
rm./
anti, jayamukti cikarang pusat
/
vete, no 62i rt 2 purus padang barat
/
jl. dr. wahidin sudi i, no 154 toko kelvinontong, kepatihan
/jl. dr.
rum diponegoro iii
/rum diponegoro iii
makam 7 kandangan benowo
/makam 7
jalan raya casablanca, apartemen casablanca mansion kav 9
/jalan raya
grogol selatan persa 3 27 rt 2 7 kebayoran lama
/grogol selatan persa 3
lalang kel. gang berkats, no 26 medan sunggal
lalang kel/
tanah merdeka, no 64a aa pizza & burger, 5 rambutan ciracas
/tanah merdeka,
duren sawit raden inten ii 3 rt 8 rw 10 13440 duren sawit
duren sawit raden inten ii/
graha situsari permai blok a7 no13
graha situsari permai/
setu kamp keramat 23 rt 7 4 13880 cipayung
/setu kamp keramat
seram baru 52 pandau hilir medan perjuangan
/seram baru
sutoyo sisw, 16 huta tonga tonga sibolga utara
/sutoyo
sat kulon 2, margahayu utara babakan ciparay
/sat kulon
kafe om kumis, kelu

/tmp trik,
gg. 1 no 45 kebon sirih rt 6 2 menteng
/gg. 1
buah segar muara buah bendu wates,
buah segar/
hard suparmi ahmad yani no 13 57558
hard suparmi ahmad yani/
masjid baitul hikmahmah aceh barat- meulaboh cot darat
masjid baitul hikmahmah aceh barat-/
tugu utara iolin shoes, bhayangkara tugu no 85 rw 1 koja
iolin shoes, bhayangkara tugu/tugu utara
gg. wari 21 pulo brayan darat ii medan timur
/gg.
toko bintangang mas
toko bintangang mas/
ganting parak gadang air camar 10c 25142
ganting parak/
perum langensari indah
perum langensari indah/
danau laut tawar, tanah abang
/danau laut
kryad muraya banyuwda aceh, t panglima polem, laksana
/kryad muraya
jl. bumi masjid raya, komplek bumi persada rt rw. 02 001. no.21. pemurus baru
/jl. bumi
welo'v gift, bota garden
/welo'v gift,
haji makri no 20 toko kacamata amanah, pekayon jaya bekasi selatan
/haji makri
pin 6 kedunghalang bogor utara - kota
/pin 6
cuci motor & karpet iswahyudi tulungagung
cuci motor & karpet iswahyudi/
rumah jahit benn 

/sewa gaun
kemanggisan angg nel 33 rt 3 1 palmerah
/kemanggisan angg nel
lamt no 55 pamulang timur kel. pamulang
/lamt
simp manak pagar alam
/simp manak pagar alam
toko bangunan safira ujung batu ujung batu
toko bangunan safira ujung batu/
bali children foundation, raya kesa, kuta utara
bali children/
lebak bulus laks, no 33 8 cilandak
lebak bulus/
pondok kelapa haji namawi 20 rt 9 2 duren sawit
/pondok kelapa haji
gun singgalang 17855 cikarang selatan
/gun singgalang
kapuk muara smp 1 56-67 rt 2 3 14460 penjaringan
/kapuk muara
sentra teknik indonesia. cv, nga tirto ii,
sentra teknik indonesia/
joko ting pondoksari nguntoronadi
joko ting/
karet flamboyan 18-2 rt 9 3 12920 setia budi
/karet flamboyan
gang tela, no 20
/gang tela,
de latinos bsd, jl. de latinos cluster hacienda mexicano blok c15 9
latinos cluster hacienda mexicano/de latinos
bpr kedu arthas, suy, kertosari temanggung
bpr kedu/
mamajang luar kel. beruang, 6 90132
mamajang luar kel/
bat ku med 86 percut sei tuan
/bat ku me

...

olah raga 1 cililitan kramat jati
/olah raga 1
nasi gorengkunge soto padang dan lotek, nasi, ujung kalak
nasi gorengkunge soto padang dan/
gg. geger 1 8 gegerkalong sukasari
/gg. geger 1
simpanggil baru mawar, 88 tampan
/simpanggil baru
raya wonoso ar - rizqy production, wonotunggal
ar - rizqy/raya wonoso
dinas badan kesa ban padang matinggi
dinas badan/
warung bebek goa, raya dukuhwuh pakis, dukuh pakis
warung bebek/
fauzi tani bibit buah salaman
fauzi tani/
taman cibad indah 1
/taman cibad indah 1
pedong bela 39 kapuk rt 6 13
/pedong bela
mini jenderal ahmad yani b1 17
mini jenderal ahmad yani/
tupa, 155 kal squad lounge dan bar, sutawinangun
/
man tirto iii no 1 venture bengkel, rw 5
/man tirto iii
jljl.sri vila nusa indah 5 cluster murai blok sf4a no.16
cluster murai/jljl.sri vila nusa indah 5
cinangka masjid al mujahidin, 7 sawangan
masjid al/cinangka
may per 3 kapuk muara penjaringan
/may per 3
kunciran nean saba 8 rt 2 5 pinang (penang)
kunciran nean saba/
hus sas negara no 29 s

/pedurungan kiduliii 230
majelis tak (mt) ihsan,
majelis tak (mt/
griya mas matahari blok g.12 desa kecomberan
griya mas matahari/
muso sal gg. 6 karang mumus
/muso sal gg. 6
lodtunduh villa lumia bali 80571 ubud
lodtunduh villa lumia/
bank rakyatyat indonesia jend. ahmad yani, suka asih
bank rakyatyat indonesia/
artwork cafe jenetallasa
artwork cafe/
sen pri cakung
/sen pri
toko agung, hasanu, junrejo
toko agung,/
asdam heri s, para, sewon
/asdam heri
sukadamai desa sukamaararad 10 rt 3 rw 8 16165 tanah sereal
/sukadamai
tambakmekar tam mek mal no 9 jalancagak
/tambakmekar tam mek mal
gg. hik 4 pasar mingguggu
/gg. hik
soekarno - hatta, sidomulyosari timur rt 4 8 marpoyan damai
/soekarno -
lampera raya 40 rt 13 rw 3 jatibening baru
/lampera raya
warung bu euis, pilang raya,
warung bu/
kapuk kapuk raya 44 rt 2 2 penjaringan
/kapuk kapuk raya
g repe 13 tampan payung sekaki
/g repe
merd, no 217 ayam bakar ibu sukardi, kebalen babelan
/
bojong rawalumbu suka, 19 rt 4 rw 41 17116 rawalumbu

/cingcinere raya
nerada estate blok b1 no. 18 rt rw.005 010
nerada estate/
ks tubun pulo dk. sugi, rt 1 rembang
dk. sugi,/ks tubun
puskesmas pega 1, reban
puskesmas pega/
candi sewu,
/candi sewu,
duta prat, bojong rawalumbu kel. rawalumbu
/duta prat,
perumahan wahana cikarang selatan
perumahan wahana/
lang no 39 kramat jati rt 6 rw 10 kramat jati
/lang
keboharan bal de no 37
/keboharan bal de
lenteng agung duren saurus pancake haji icang, 76 rw 2 jagakarsa
duren saurus pancake haji/lenteng agung
aria santika no 44 rt 3 3 karawaci
/aria santika
dok cipto, mojokampung bojonegoro
/dok cipto,
apo ananda farma, jenderal ahmad yani, seberang ulu ii
/apo ananda
raya timb, pertima karang asem
/raya timb,
bank mandiriiri raya lapan, cibogo
bank mandiriiri raya/
kantor desa klapa sawit, inp, klapasawit purwojati
kantor desa klapa/
t ham bendahara sd neg arun
sd neg arun/t ham bendahara
kali raya kalimatigaisari pasar rebo
/kali raya
perumahan griya pes, cibalongsari klari
perumahan griya/
solokp

fad sei/pon buah
sat tower buil profe doktor sat timur kuningan
sat tower buil profe doktor sat timur/
coto maka marannu, d i panja, sungai pinang
/coto
rm. ibu, lin sumateatera sor aur gading
rm./
gang sos, pasar mingguggu kel.
/gang sos,
petemon petemon ii a 104 9 60252
/petemon petemon ii a
erze fashion, mar 1, purwokerto wetan
/erze fashion,
pradahkali kendal kel. darmo per selatan xii no 32 rw 6 60226
kel. darmo per selatan xii/pradahkali kendal
belakang rth bajulmati rt rw 04 02 karang anyar
/belakang rth
h o s cokroaminotooam,
/h o s
olym cell, ki hajar dewantara,
/olym
bank mandiriiri, ki hajar dewantara,
bank mandiriiri,/
mut
/mut
smp negeri 2 banjarjo, sura,
smp negeri 2/
dewa putu kerta pemaron buleleng
dewa putu/
anna souvenir,
/anna
war - cir 109 sukahaji tegal waru
/war - cir
j prayi gg. pray, gedong pasar rebo
/j prayi gg
sion text maka, datuk riban tallo
/sion text
alfamart gatsu 2 jend. gatot subroto cilacap tengah
/alfamart gatsu 2
del 30 cileungsi kidul
/del
masjid p

/a sal
kapas baru v 3 gading 7 tambaksari
/kapas baru v
dk. wijaya rt 1 rw 1 (selepan lor lapangan kopen), kel kopen
dk./
b.nu borob no 1 lowokwaru
/b.nu borob
raya sibang kaja 6 pawon wong solo sibang kaja
pawon wong solo/raya sibang kaja 6
pal raya, bas rahmat,
/pal raya,
perumahan tni al griya samudra asri e4 01 kec.taman kab.sidoarjo
perumahan tni al griya samudra asri/
bubu multi sejaht pt yunus sanis, kebun handil
bubu multi sejaht pt yunus/
sd negeri 022iii lempur danau, danau kerinci,
sd negeri 022iii lempur/
utan panjangang kel. utan panjangang, no 19 6 10650
/utan
surya cell, perum pondok surya karang tengah
/surya cell,
seko tekn gang ii kranji purwokerto timur
/seko tekn gang ii
krenchise fried chicken pasir impun,
krenchise fried chicken/
bendogerit pramb 23 66133 sanan wetan
/bendogerit pramb
sawah besar 3, gayamsari
/sawah besar
raya sunan gunung jati azefacs cuci salju, jatimerta
azefacs cuci/raya sunan
gajahmungkur kal no 588 4 gajah mungkur
/gajahmungkur kal
talang pa

In [24]:
import datetime
submit = pd.DataFrame()
submit['POI/street'] = results
submit['id'] = range(submit.shape[0])

columns_titles = ["id", "POI/street"]
df_reorder=submit.reindex(columns=columns_titles)

df_reorder.to_csv(f'./scl2021_submission.csv', index = False)
print(df_reorder.head())

   id     POI/street
0   0     /s. parman
1   1  /anggrek per,
2   2    /asma laun,
3   3      ud agung/
4   4    /cut mutia,
